In [103]:
import pandas as pd

dataframe = pd.read_csv("lab_1.csv", sep=';')

dataframe.head()

,id,name,price,currency,created_at
0,SKU-1001,shoes,799,SEK,2024-01-10
1,SKU-1002,pants,520,SEK,2024/02/15
2,SKU-1003,shirts,450,SEK,NaN
3,NaN,jacket,-200,SEK,2024-13-01
4,SKU-1005,NaN,NaN,NaN,


In [104]:
dataframe['price'] = pd.to_numeric(dataframe['price'], errors='coerce')
dataframe['created_at'] = dataframe['created_at'].str.replace("/", "-", regex=False)
dataframe['created_at'] = pd.to_datetime(dataframe['created_at'], errors='coerce')
dataframe['name'] = dataframe['name'].str.strip()
dataframe['currency'] = dataframe['currency'].str.strip()

dataframe.head()

,id,name,price,currency,created_at
0,SKU-1001,shoes,799.0,SEK,2024-01-10
1,SKU-1002,pants,520.0,SEK,2024-02-15
2,SKU-1003,shirts,450.0,SEK,NaT
3,NaN,jacket,-200.0,SEK,NaT
4,SKU-1005,NaN,NaN,NaN,NaT


In [127]:
dataframe['missing_value'] = (
    (dataframe['id'].isna()) |
    (dataframe['name'].isna()) |
    (dataframe['price'].isna()) |
    (dataframe['currency'].isna()) |
    (dataframe['created_at'].isna())
)

dataframe['sus_price'] = (
    (dataframe['price'].isna()) |
    (dataframe['price'] < 0) |
    (dataframe['price'] > 1000)
)

dataframe.head()

,id,name,price,currency,created_at,missing_value,sus_price
0,SKU-1001,shoes,799.0,SEK,2024-01-10,False,False
1,SKU-1002,pants,520.0,SEK,2024-02-15,False,False
2,SKU-1003,shirts,450.0,SEK,NaT,True,False
3,NaN,jacket,-200.0,SEK,NaT,True,True
4,SKU-1005,NaN,NaN,NaN,NaT,True,True


In [130]:
reject_condition = (
       (dataframe['name'] == 'cap') |
       (dataframe["price"] == '300')
)
df_rejected = dataframe[reject_condition].copy()
df_valid = dataframe[~reject_condition].copy()

df_rejected.to_csv('rejected_products.csv',sep=',', index=False, header=True)

df = pd.read_csv('rejected_products.csv', sep=',')
df

,id,name,price,currency,created_at,missing_value,sus_price
0,SKU-1013,cap,75.0,SEK,2024-04-01,False,False
1,SKU-1053,cap,85.0,SEK,2025-01-15,False,False


In [131]:
data = {
    "price_average": [dataframe['price'].mean()],
    "price_median": [dataframe['price'].median()],
    "products_frequency": [len(dataframe)],
    "missing_price": [dataframe['price'].count()]
}

analytics_summary = pd.DataFrame(data)

analytics_summary.to_csv('analytics_summary.csv',sep=',', index=False)
df = pd.read_csv('analytics_summary.csv', sep=',')